In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime
import pandas as pd
import pandas as pd
#Punctuations
import string
#reg Expressions
import re
#emojis with description
import demoji
#sentences to words
from nltk.tokenize import TreebankWordTokenizer
#paras to sentences
from nltk.tokenize import sent_tokenize
#stopwords
from nltk.corpus import stopwords
#for converting word to vector
import gensim.models.word2vec as w2v
from sklearn.manifold import TSNE

In [ ]:
%%time
tweet_data = pd.read_csv("merged_csv_en_rs_it.csv")

In [ ]:
leftTroll_data =  tweet_data[(tweet_data['account_category'] == 'LeftTroll')]

In [ ]:
leftTroll_data['date'] = pd.to_datetime(leftTroll_data['publish_date'])
leftTroll_data['YEAR']=pd.to_datetime(leftTroll_data.date,format='%Y-%m-%d %H:%M:%S').dt.year
leftTroll_data['MONTH']=pd.to_datetime(leftTroll_data.date,format='%Y-%m-%d %H:%M:%S').dt.month
leftTroll_data['DAY']=pd.to_datetime(leftTroll_data.date,format='%Y-%m-%d %H:%M:%S').dt.day
leftTroll_data2016 = leftTroll_data[(leftTroll_data['YEAR'] == 2016)]
leftTroll_data2017 = leftTroll_data[(leftTroll_data['YEAR'] == 2017)]
leftTroll_data2018 = leftTroll_data[(leftTroll_data['YEAR'] == 2018)]

In [ ]:
chron2016=pd.DataFrame(leftTroll_data2016.groupby(['MONTH','DAY']).count()).reset_index().rename(columns={'content':'Total Tweet'})[['MONTH','DAY','Total Tweet']]
chron2016=chron2016.pivot('DAY','MONTH','Total Tweet')
chron2017=pd.DataFrame(leftTroll_data2017.groupby(['MONTH','DAY']).count()).reset_index().rename(columns={'content':'Total Tweet'})[['MONTH','DAY','Total Tweet']]
chron2017=chron2017.pivot('DAY','MONTH','Total Tweet')
chron2018=pd.DataFrame(leftTroll_data2018.groupby(['MONTH','DAY']).count()).reset_index().rename(columns={'content':'Total Tweet'})[['MONTH','DAY','Total Tweet']]
chron2018=chron2018.pivot('DAY','MONTH','Total Tweet')

In [ ]:
plt.figure(figsize=(12,10))
plt.title('Variation in Number of tweets per day for Left Troll in the year 2016', fontsize=14)
sns.heatmap(chron2016,annot=True,fmt='g', cmap='viridis',linecolor='grey',linewidths=0.06)

In [ ]:
plt.figure(figsize=(12,10))
plt.title('Variation in Number of tweets per day for Left Troll in the year 2017', fontsize=14)
sns.heatmap(chron2017,annot=True,fmt='g', cmap='seismic',linecolor='grey',linewidths=0.06)

In [ ]:
#plt.figure(figsize=(12,10))
#plt.title('Variation in Number of tweets per day for Left Troll in the year 2018', fontsize=14)
#sns.heatmap(chron2018,annot=True,fmt='g', cmap='viridis',linecolor='grey',linewidths=0.06)

In [ ]:
leftTroll_dataEnglish = leftTroll_data[(leftTroll_data['language'] == 'English')]

In [ ]:
def extract_hashtags(tweet):
    hashtags_list = []
    if len(re.findall("(#[^#\s]+)", tweet)) > 0:
        hashtags_list.append(re.findall("(#[^#\s]+)", tweet))
    else:
        hashtags_list.append(["0"])
    return hashtags_list[0]

In [ ]:
all_hashtags = leftTroll_data['content'].map(extract_hashtags).tolist()

In [ ]:
oneList_hashtags = []
for i in all_hashtags:
    for j in i:
            oneList_hashtags.append(j)

In [ ]:
oneList_hashtags_series = pd.Series(oneList_hashtags)
hashtag_counts = oneList_hashtags_series.value_counts()

In [ ]:
# Get hashtag terms from the series and convert to list
hashes = oneList_hashtags_series.values
hashes = hashes.tolist()

# convert list to one string with all the words
hashes_words = " ".join(hashes)

# generate the wordcloud. the max_words argument controls the number of words on the cloud
from wordcloud import WordCloud
wordcloud = WordCloud(width= 1600, height = 800, 
                      relative_scaling = 1.0, 
                      colormap = "Blues",
                     max_words = 100).generate(hashes_words)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# plot data
fig, ax = plt.subplots(figsize=(15,7))
# use unstack()
leftTroll_data.groupby(['author','region']).count()['content'].unstack().plot(ax=ax)

In [ ]:
from collections import Counter

def getRetweets(text):
    for string in text:
        if isinstance(string,str):
            tokens = string.split(' ')
            if tokens[0] == 'RT' and tokens[1].startswith('@'):
                yield tokens[1]


retweets = getRetweets(leftTroll_dataEnglish['content'])
retweet_frequency = Counter(retweets)
#retweet_frequency.most_common(100)

In [ ]:
plt.figure(figsize=(15,10))
top30 = retweet_frequency.most_common(30)
keys = [x[0] for x in top30]
vals = [x[1] for x in top30]
xpos = range(len(vals),0,-1)
plt.barh(xpos, vals)
plt.yticks(xpos,keys)
plt.title("Retweets")

plt.show()